# Multi-AOI Vegetation Indices with Dask (NDVI, EVI, SAVI)

This notebook loads four small AOIs across Australia and computes NDVI/EVI/SAVI time-series using ODC-STAC with Dask. It then clips each AOI and plots/export zonal statistics for all polygons.

## 1) Install packages

In [ ]:
!conda activate py310

In [2]:
%pip install -q pystac-client planetary-computer odc-stac xarray geopandas rioxarray rasterio matplotlib dask[distributed]

Note: you may need to restart the kernel to use updated packages.


## 2) Imports and Dask

In [2]:
from pystac_client import Client as StacClient
import planetary_computer
import xarray as xr
from odc.stac import stac_load
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
from shapely.geometry import shape, mapping
from dask.distributed import Client as DaskClient, LocalCluster
import warnings
warnings.filterwarnings('ignore')
xr.set_options(keep_attrs=True);
cluster = LocalCluster(n_workers=16, threads_per_worker=32)
client = DaskClient(cluster); client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 512,Total memory: 55.95 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63915,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 512
Started: Just now,Total memory: 55.95 GiB
Comm: tcp://127.0.0.1:63994,Total threads: 32
Dashboard: http://127.0.0.1:64021/status,Memory: 3.50 GiB
Nanny: tcp://127.0.0.1:63918,


## 3) STAC catalog

In [5]:
catalog = StacClient.open('https://planetarycomputer.microsoft.com/api/stac/v1/', modifier=planetary_computer.sign_inplace)
catalog

<Client id=microsoft-pc>

## 4) Load multi-AOI GeoJSON and bounding box

In [11]:
aoi_path = 'australia_4_polygons.geojson'
# aoi_path = '../data/forest283.geojson'
gdf = gpd.read_file(aoi_path).to_crs(4326)
# Dissolve for bbox search but keep individual polygons for zonal stats
bbox = gdf.dissolve().total_bounds.tolist()
gdf

,id,name,note,geometry
0,1,WA_site,~9.6 ha square,"POLYGON ((119.4985 -23.7015, 119.5015 -23.7015..."
1,2,NT_site,~9.6 ha square,"POLYGON ((133.8985 -18.5015, 133.9015 -18.5015..."
2,3,QLD_site,~9.6 ha square,"POLYGON ((144.1985 -22.0015, 144.2015 -22.0015..."
3,4,NSW_site,~9.6 ha square,"POLYGON ((146.9985 -32.5015, 147.0015 -32.5015..."


## 5) Search Landsat 8 C2 L2 (2015–2024) <20% cloud

In [8]:
results = catalog.search(
    collections=['landsat-c2-l2'],
    bbox=bbox,
    datetime=['2024-06-01', '2024-12-31'],
    query={'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['landsat-8']}}
)
items = list(results.get_items())
len(items), items[0] if items else None

(11, <Item id=LC08_L2SP_090078_20241025_02_T1>)

## 6) Load data cube (blue, red, nir) with chunks

In [ ]:
bands = ['blue','red','nir08']
chunks = {'time': 1, 'x': 1024, 'y': 1024}
data = stac_load(items=items, bands=bands, bbox=bbox, groupby='time', chunks=chunks)
data

<xarray.Dataset> Size: 24TB
Dimensions:      (y: 60451, x: 98822, time: 662)
Coordinates:
  * y            (y) float64 484kB -2.049e+06 -2.049e+06 ... -3.862e+06
  * x            (x) float64 791kB 1.302e+05 1.302e+05 ... 3.095e+06 3.095e+06
    spatial_ref  int32 4B 32651
  * time         (time) datetime64[ns] 5kB 2024-10-01T01:30:47.875149 ... 202...
Data variables:
    blue         (time, y, x) uint16 8TB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    red          (time, y, x) uint16 8TB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    nir08        (time, y, x) uint16 8TB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>

## 7) Aggregate to monthly (median)

In [ ]:
data_monthly = data.resample(time='1M').median()
data_monthly

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103ab70e0>>
Traceback (most recent call last):
  File "/Users/ben/miniconda3/envs/sklearn-env/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
2025-08-11 12:22:56,166 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/Users/ben/miniconda3/envs/sklearn-env/lib/python3.13/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/Users/ben/miniconda3/envs/sklearn-env/lib/python3.13/asyncio/base_events.py", line 725, in run_until_complete
    return future.result()
           ~~~~~~~~~~~~~^^
  File "/Users/ben/miniconda3/envs/sklearn-env/lib/python3.13/site-packages/distributed/nanny.py"

## 8) Compute indices (NDVI, EVI, SAVI)

In [ ]:
nir = data_monthly['nir08']; red = data_monthly['red']; blue = data_monthly['blue']
NDVI = (nir - red) / (nir + red)
EVI = 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0)
SAVI = (1.0 + 0.5) * (nir - red) / (nir + red + 0.5)
NDVI, EVI, SAVI

(<xarray.DataArray (time: 3, y: 60451, x: 98822)> Size: 143GB
 dask.array<truediv, shape=(3, 60451, 98822), dtype=float64, chunksize=(1, 466, 466), chunktype=numpy.ndarray>
 Coordinates:
   * y            (y) float64 484kB -2.049e+06 -2.049e+06 ... -3.862e+06
   * x            (x) float64 791kB 1.302e+05 1.302e+05 ... 3.095e+06 3.095e+06
     spatial_ref  int32 4B 32651
   * time         (time) datetime64[ns] 24B 2024-10-31 2024-11-30 2024-12-31
 Attributes:
     nodata:   0,
 <xarray.DataArray (time: 3, y: 60451, x: 98822)> Size: 143GB
 dask.array<truediv, shape=(3, 60451, 98822), dtype=float64, chunksize=(1, 466, 466), chunktype=numpy.ndarray>
 Coordinates:
   * y            (y) float64 484kB -2.049e+06 -2.049e+06 ... -3.862e+06
   * x            (x) float64 791kB 1.302e+05 1.302e+05 ... 3.095e+06 3.095e+06
     spatial_ref  int32 4B 32651
   * time         (time) datetime64[ns] 24B 2024-10-31 2024-11-30 2024-12-31
 Attributes:
     nodata:   0,
 <xarray.DataArray (time: 3, y: 60451,

: 

## 9) Zonal stats per AOI and combined plotting

In [ ]:
# Clip each AOI and compute means over x/y
from shapely.geometry import mapping as shp_mapping
aoi_crs = gdf.crs or 'EPSG:4326'
records = []
for idx, row in gdf.iterrows():
    name = row.get('name', f'AOI_{idx}')
    geoms = [shp_mapping(row.geometry)]
    ndvi_m = NDVI.rio.clip(geoms, aoi_crs, drop=True).mean(dim=['y','x']).rename('NDVI')
    evi_m = EVI.rio.clip(geoms, aoi_crs, drop=True).mean(dim=['y','x']).rename('EVI')
    savi_m = SAVI.rio.clip(geoms, aoi_crs, drop=True).mean(dim=['y','x']).rename('SAVI')
    ds = xr.concat([ndvi_m, evi_m, savi_m], dim='metric').assign_coords(metric=['NDVI','EVI','SAVI'])
    ds = ds.to_series().reset_index(name='value').rename(columns={'metric':'metric'})
    ds['aoi'] = name
    records.append(ds)
import pandas as pd
df = pd.concat(records, ignore_index=True)
# Plot: one figure with lines for each AOI per metric
fig, ax = plt.subplots(figsize=(10,6))
for metric in ['NDVI','EVI','SAVI']:
    pivot = df[df.metric==metric].pivot(index='time', columns='aoi', values='value')
    pivot.plot(ax=ax, linewidth=1.2, alpha=0.8)
ax.set_title('AOI Vegetation Indices (Monthly Means)')
ax.set_ylabel('Index value')
plt.legend(title='AOI', bbox_to_anchor=(1.02,1), loc='upper left')
plt.tight_layout()
df.head()

## 10) Export combined CSV

In [ ]:
df.to_csv('AOI_multi_indices_timeseries.csv', index=False)
'Wrote AOI_multi_indices_timeseries.csv'

### Optional: wide-format CSVs and clearer plots

- Export wide CSVs per metric (rows=time, columns=AOIs).
- Export a single wide CSV with MultiIndex columns (metric, AOI).
- Provide per-metric subplots for a cleaner visualization.


In [ ]:
# Build wide-format tables per metric (rows=time, columns=AOIs)
import pandas as pd

metrics = ['NDVI','EVI','SAVI']
wide_tables = {}
for metric in metrics:
    wide = df[df.metric == metric].pivot(index='time', columns='aoi', values='value')
    wide_tables[metric] = wide
    wide.to_csv(f'AOI_{metric}_timeseries_wide.csv', index_label='time')

# Also export a single CSV with a two-level column index (metric, aoi)
wide_multi = pd.concat(wide_tables, axis=1)
wide_multi.to_csv('AOI_all_metrics_timeseries_wide_multiindex.csv', index_label='time')

# Per-metric subplots for clearer visualization
fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
for ax, metric in zip(axes, metrics):
    wide_tables[metric].plot(ax=ax)
    ax.set_title(f'{metric} (Monthly Means)')
    ax.set_ylabel(metric)
    ax.legend(title='AOI', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
'Exported wide-format CSVs and rendered per-metric plots'

## 6) Polygon-by-polygon processing (memory-safe, Dask-friendly)

Instead of loading data for the dissolved bounding box of all AOIs (which can be huge),
this section iterates over each polygon:
- Search STAC only within that polygon's bbox
- Load with ODC-STAC using Dask chunks
- Aggregate monthly and compute NDVI/EVI/SAVI
- Clip to the polygon and compute spatial means
- Append to a combined results table

This approach reduces memory pressure and avoids building a massive Dask graph.
You can skip the previous bulk loading/aggregation cells if you use this section.


In [12]:
# Polygon-by-polygon processing (memory-safe, Dask-friendly)

# Parameters for search and chunks
# collections = ['landsat-c2-l2']
collections = ['sentinel-2-l2a']
datetime_range = ['2015-06-01', '2024-12-31']
cloud_filter = {'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['sentinel-2']}} # Updated platform
bands = ['blue','red','nir08']
chunks = {'time': 1, 'x': 4196, 'y': 4196}

# Container for all AOI results
import pandas as pd
import xarray as xr
import rioxarray
from shapely.geometry import mapping as shp_mapping
from tqdm.auto import tqdm # Import tqdm

records_pp = []

# Iterate polygons with tqdm progress bar
for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Processing AOIs"):
    aoi_name = row.get('name', f'AOI_{idx}')
    geom = row.geometry
    # Per-AOI bbox search
    minx, miny, maxx, maxy = geom.bounds
    aoi_bbox = [minx, miny, maxx, maxy]

    # print(f"Processing AOI: {aoi_name}") # Commenting out for cleaner tqdm output

    # Search STAC for this AOI only
    print(f"Searching STAC for {aoi_name} with bbox: {aoi_bbox}, datetime: {datetime_range}, collections: {collections}, query: {cloud_filter}") # Debug print
    res = catalog.search(collections=collections, bbox=aoi_bbox, datetime=datetime_range, query=cloud_filter)
    items_aoi = list(res.get_items())
    print(f"Found {len(items_aoi)} items for {aoi_name}") # Debug print

    if not items_aoi:
        # print(f'No items found for {aoi_name} in the specified time range; skipping') # Commenting out for cleaner tqdm output
        continue

    # Load data lazily with Dask for the AOI
    try:
        data_aoi = stac_load(items=items_aoi, bands=bands, bbox=aoi_bbox, groupby='time', chunks=chunks)
    except Exception as e:
        print(f"Error loading data for {aoi_name}: {e}; skipping")
        continue

    if data_aoi is None or len(data_aoi.time) == 0:
        # print(f'No data loaded for {aoi_name}; skipping') # Commenting out for cleaner tqdm output
        continue

    # Monthly aggregation
    dm = data_aoi.resample(time='1M').median()

    # Compute indices
    if 'nir08' in dm and 'red' in dm and 'blue' in dm:
        nir = dm['nir08']; red = dm['red']; blue = dm['blue']
        # Compute all indices at once
        ndvi = (nir - red) / (nir + red)
        evi = 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0)
        savi = (1.0 + 0.5) * (nir - red) / (nir + red + 0.5)

        # Combine indices into a single dataset
        indices_ds = xr.Dataset({'NDVI': ndvi, 'EVI': evi, 'SAVI': savi})

        # Ensure CRS is set for clipping
        aoi_crs = gdf.crs or 'EPSG:4326'
        if indices_ds.rio.crs is None:
             # print(f'Indices dataset has no CRS, setting to match AOI CRS: {aoi_crs}') # Commenting out for cleaner tqdm output
             indices_ds = indices_ds.rio.write_crs(aoi_crs)
        elif indices_ds.rio.crs != aoi_crs:
             print(f'Indices dataset CRS ({indices_ds.rio.crs}) does not match AOI CRS ({aoi_crs}). Reprojecting.')
             indices_ds = indices_ds.rio.reproject(aoi_crs)

        # Clip the combined indices dataset to the polygon and compute spatial means
        try:
            geoms = [shp_mapping(geom)]
            # Clip and compute mean for all indices at once
            zonal_stats_ds = indices_ds.rio.clip(geoms, aoi_crs, drop=True, invert=False).mean(dim=['y','x'])

            # Convert to DataFrame and append AOI name
            df_aoi = zonal_stats_ds.to_series().reset_index(name='value').rename(columns={'variable':'metric'})
            df_aoi['aoi'] = aoi_name
            records_pp.append(df_aoi)
            # print(f"Successfully processed {aoi_name}") # Commenting out for cleaner tqdm output

        except Exception as e:
            print(f"Error clipping or computing zonal statistics for {aoi_name}: {e}; skipping")
            continue
    else:
        # print(f'Missing required bands in data for {aoi_name}; skipping index computation') # Commenting out for cleaner tqdm output
        continue


# Combine all AOIs results
if records_pp:
    df_pp = pd.concat(records_pp, ignore_index=True)
    # Save both long and wide formats
    df_pp.to_csv('AOI_multi_indices_timeseries_polygon_by_polygon_optimized_long.csv', index=False)
    # Wide per metric
    for metric in ['NDVI','EVI','SAVI']:
        wide = df_pp[df_pp.metric==metric].pivot(index='time', columns='aoi', values='value')
        wide.to_csv(f'AOI_{metric}_timeseries_polygon_by_polygon_optimized_wide.csv', index_label='time')
    # Combined wide with MultiIndex columns
    wide_multi = pd.concat({m: df_pp[df_pp.metric==m].pivot(index='time', columns='aoi', values='value') for m in ['NDVI','EVI','SAVI']}, axis=1)
    wide_multi.to_csv('AOI_all_metrics_polygon_by_polygon_optimized_wide_multiindex.csv', index_label='time')
    print('Optimized polygon-by-polygon export complete.')
else:
    print('No AOI results generated from optimized processing.')

Processing AOIs:   0%|          | 0/4 [00:00<?, ?it/s]

Searching STAC for WA_site with bbox: [119.4985, -23.7015, 119.5015, -23.6985], datetime: ['2015-06-01', '2024-12-31'], collections: ['sentinel-2-l2a'], query: {'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['sentinel-2']}}
Found 0 items for WA_site
Searching STAC for NT_site with bbox: [133.8985, -18.5015, 133.9015, -18.4985], datetime: ['2015-06-01', '2024-12-31'], collections: ['sentinel-2-l2a'], query: {'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['sentinel-2']}}
Found 0 items for NT_site
Searching STAC for QLD_site with bbox: [144.1985, -22.0015, 144.2015, -21.9985], datetime: ['2015-06-01', '2024-12-31'], collections: ['sentinel-2-l2a'], query: {'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['sentinel-2']}}
Found 0 items for QLD_site
Searching STAC for NSW_site with bbox: [146.9985, -32.5015, 147.0015, -32.4985], datetime: ['2015-06-01', '2024-12-31'], collections: ['sentinel-2-l2a'], query: {'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['sentinel-2']}}
Found 0 ite

## 7) Plot time-series for all AOIs and indices

This plots NDVI, EVI, and SAVI as separate time-series subplots, with one line per AOI.
It uses the polygon-by-polygon results if available (df_pp), otherwise falls back to the earlier df.


In [13]:
# Choose the results DataFrame: prefer polygon-by-polygon (df_pp), else fallback to df
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline

data_src = None
try:
    # df_pp is produced by polygon-by-polygon section
    if isinstance(df_pp, pd.DataFrame) and {'time','aoi','metric','value'}.issubset(df_pp.columns):
        data_src = df_pp.copy()
except NameError:
    pass

if data_src is None:
    try:
        if isinstance(df, pd.DataFrame) and {'time','aoi','metric','value'}.issubset(df.columns):
            data_src = df.copy()
    except NameError:
        pass

if data_src is None:
    raise RuntimeError('No results DataFrame found. Run either the polygon-by-polygon section or the earlier zonal stats block first.')

# Ensure time is datetime and remove zero values
data_src['time'] = pd.to_datetime(data_src['time'])
data_src = data_src[data_src['value'] != 0].copy()

metrics = ['NDVI', 'EVI', 'SAVI']
fig, axes = plt.subplots(len(metrics), 1, figsize=(11, 10), sharex=True)

for ax, metric in zip(axes, metrics):
    metric_data = data_src[data_src.metric == metric]
    for aoi in metric_data['aoi'].unique():
        aoi_data = metric_data[metric_data['aoi'] == aoi].sort_values(by='time')

        if len(aoi_data) > 1:
            # Convert datetime to numerical representation (e.g., timestamps) for interpolation
            x = aoi_data['time'].apply(lambda t: t.timestamp()).values
            y = aoi_data['value'].values

            # Create spline
            X_Y_Spline = make_interp_spline(x, y)

            # Create evenly spaced time points for smoothing
            X_ = np.linspace(x.min(), x.max(), 500)
            Y_ = X_Y_Spline(X_)

            # Convert numerical time points back to datetime
            time_smooth = pd.to_datetime(X_, unit='s')

            ax.plot(time_smooth, Y_, linewidth=1.5, label=aoi)
        elif len(aoi_data) == 1:
             # Plot single point if only one data point exists
             ax.plot(aoi_data['time'], aoi_data['value'], 'o', label=aoi)


    ax.set_title(f'{metric} time-series by AOI (Smoothed)')
    ax.set_ylabel(metric)
    ax.grid(True, alpha=0.2)
    ax.legend(title='AOI', bbox_to_anchor=(1.02, 1), loc='upper left')

axes[-1].set_xlabel('Time')
plt.tight_layout()
'Rendered per-metric time-series plots for all AOIs (Smoothed)'

RuntimeError: No results DataFrame found. Run either the polygon-by-polygon section or the earlier zonal stats block first.